# 대통령 세월호 담화문 분석

## 사건발생 34일째의 담화문

#### 담화문 내용 살펴보기
```
!cat ../data/statement.txt
```

#### 담화문 로드해서 라인 별로 자르기
```python
with open('../data/statement.txt', 'r') as f:
    lines = f.read().splitlines()
```

#### 빈 문장을 정리하기
```python
sents = [line for line in lines if line != '']
```

#### 형태소 분석기 테스트
문장 리스트 `sents`의 첫 번째 문장을 가져와서 코모란 형태소 분석기로 분석해본다.
```python
sent = sents[0]
print(sent)

from konlpy.tag import Komoran
tagger = Komoran()
tags = tagger.pos(sent)
print(tags)
```

#### 각 문장 형태소 분석
```python
sents = [tagger.pos(sent) for sent in sents]
```

#### 한 번이라도 등장한 명사 종류 수집하기
각 문장의 형태소 중 일반명사(NNG)또는 고유명사(NNP)를 수집한다. 명사 목록을 만들 것이기 때문에 집합을 사용해서 명사를 모은 후 리스트로 변환한다. 그리고 문장-단어 행렬 계산에 사용할 명사-번호 딕셔너리도 준비한다.
```python
nouns = set()

for sent in sents:
    for morph, tag in sent:
        if tag in ['NNP', 'NNG']:
            nouns.add(morph)
            
nouns = list(nouns)
noun_index = {noun: i for i, noun in enumerate(nouns)}
```

### 문장-단어 행렬 계산
```python
import numpy as np
occurs = np.zeros([len(sents), len(nouns)])

for i, sent in enumerate(sents):
    for morph, tag in sent:
        if tag in ['NNP', 'NNG']:
            index = noun_index[morph]
            occurs[i][index] = 1
```

### 공존 단어 행렬 계산
```python
co_occurs = occurs.T.dot(occurs)
```

### 연관 단어 NetworkX 그래프 생성
```python
import networkx as nx
G = nx.Graph()

for i in range(len(nouns)):
    for j in range(i + 1, len(nouns)):
        if co_occurs[i][j] > 3:
            G.add_edge(nouns[i], nouns[j])
```

### 그래프 시각화
```python
%matplotlib inline
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 15))
layout = nx.spring_layout(G, k=.1)
nx.draw(G, pos=layout, with_labels=True, \
        font_size=20, font_family="AppleGothic", \
        alpha=0.3, node_size=2000)
plt.show()
```

### 1주기 담화문으로 반복하기

#### 파일 로드, 빈 줄 정리
```python
with open('../data/statement_next.txt', 'r') as f:
    lines = f.read().splitlines()
lines = [line for line in lines if line != '']
```

#### 각 줄을 문장으로 쪼개 문장 리스트에 삽입
```python
from nltk.tokenize import sent_tokenize

sents = []
for line in lines:
    line_sents = sent_tokenize(line)
    sents.extend(line_sents)
```

#### 각 문장을 형태소 분석
```python
from konlpy.tag import Komoran

tagger = Komoran()
sents = [tagger.pos(line) for line in sents]
```

#### 명사 리스트 추출
```python
nouns = set()

for sent in sents:
    for morph, tag in sent:
        if tag in ['NNP', 'NNG']:
            nouns.add(morph)

nouns = list(nouns)
noun_index = {noun: i for i, noun in enumerate(nouns)}
```

#### 문장-단어 행렬 및 공존 단어 행렬 계산
```python
occurs = np.zeros([len(sents), len(nouns)])

for i, sent in enumerate(sents):
    for morph, tag in sent:
        if tag in ['NNP', 'NNG']:
            index = noun_index[morph]
            occurs[i][index] = 1

co_occurs = occurs.T.dot(occurs)
```

#### 연관 단어 그래프 생성
```python
import networkx as nx
g = nx.Graph()

for i in range(len(nouns)):
    for j in range(i + 1, len(nouns)):
        if co_occurs[i][j] > 1:
            g.add_edge(nouns[i], nouns[j])
```

#### 그래프 시각화
```python
plt.figure(figsize=(15, 15))
layout = nx.spring_layout(g, k=.1)
nx.draw(g, pos=layout, with_labels=True, font_size=20, font_family="AppleGothic", alpha=0.3, node_size=2000)
plt.show()
```